# Data Visualization in Python
----

# [matplotlib](https://matplotlib.org/) and pandas

pandas uses matplotlib to make plotting data sets easy.  You can also use features of matplotlib directly when plotting. We're going to use both methods below to see the differences and similarities.  The method (pandas or matplotlib) is marked for the cells so you can keep track of the different methods.

## Parts of a matplotlib plot

<img src="https://github.com/nuitrcs/pythonworkshops/raw/master/dataanalysis/visualization/plotparts.png" width=500>

<img src="https://github.com/nuitrcs/pythonworkshops/raw/master/dataanalysis/visualization/anatomy1.png" width=500>

## matplotlib documentation 

matplotlib.org

* http://matplotlib.org/users/beginner.html
* http://matplotlib.org/api/pyplot_summary.html
* http://matplotlib.org/gallery.html 

## Getting Started

Import libraries, tell the notebook to show matplotlib plots inline and make them interactive (use `%matplotlib notebook` instead for interactive plots).  Note that with interactive plots, you'll need to use plt.close() to close them afterward.

In [ ]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

import datetime as dt
import matplotlib.dates as dates

Read in [earthquakes data](http://www.ldeo.columbia.edu/~felixw/NCAeqDD/).  It's on the large side, so it will take a minute to load.

Note that there's a bunch of extra info at the start of the file, so we'll need to skip that.  Also, some columns are combined under a single header, so we'll manually specify the header.

In [ ]:
df = pd.read_csv("http://www.ldeo.columbia.edu/~felixw/NCAeqDD/NCAeqDD.v201112.1.gz", 
                 compression='gzip',
                 delim_whitespace=True, skiprows=79, na_values=[-1],
                 names=["YEAR", "MONTH", "DAY", "HOUR", "MINUTE", "SECOND",
                        "LAT", "LON", "DEP", "EH1", "EH2", "AZ", "EZ", "MAG", "ID"])
df.head(10)

Let's plot magnitude vs. depth, using only the first 50 obs.  See what different options do.

Let's just plot the data frame directly to start, using pandas defaults

In [ ]:
#pandas
df[:50].plot(x='MAG', y='DEP')

Hmm, not really what we want.  

Line plot is the default (see https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html), but we can change that with the [`kind`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html) parameter.  

Also note that we can omit the `<matplotlib.axes._subplots.AxesSubplot at 0x113d62898>` type output by putting a ; at the end of the plotting line

In [ ]:
#pandas
df[:50].plot(x='MAG', y='DEP', kind='scatter'); # semicolon suppresses printing object info

Note that pandas conveniently gave us a default legend and axis labels.

We could also use matplotlib directly.  `plt` is the shortcut for `matplotlib.pyplot` that we imported above.  See https://matplotlib.org/api/pyplot_api.html.  `plot` will make a line plot.  We use the `scatter` function for a scatter plot.

In [ ]:
#matplotlib
plt.scatter(df['MAG'][:50], df['DEP'][:50]);

We lost the default axis labels that pandas gave us.   

Now change the [plot marker](https://matplotlib.org/api/markers_api.html).  We can add the `marker` option to either pandas' `DataFrame.plot` or matplotlib's `scatter`.  

In [ ]:
# pandas
df[:50].plot(x='MAG', y='DEP', kind="scatter", marker="v");

We can also change the marker size

In [ ]:
# matplotlib
plt.scatter(df['MAG'][:50], df['DEP'][:50], marker="v", s=150); 

Now make each marker size a function of the magnitude (not a great idea from a visualization perspective...but we'll do a better example next)

In [ ]:
# matplotlib
plt.scatter(df['MAG'][:50], df['DEP'][:50], marker="v", s=[10+100*df['MAG'][:50]]);

## Colorbar legend

Make a subset of the data with only earthquakes of magnitude 5 or more

In [ ]:
bigones = df[df['MAG'] >= 5]

Color markers by year of earthquake, add axis labels

In [ ]:
#pandas
bigones.plot(x='MAG', y='DEP', # data
                   kind='scatter', # type of plot
                   s=50, # marker size
                   cmap="cool", # color scheme: https://matplotlib.org/users/colormaps.html
                   c=1.0*(bigones['YEAR']-1983)/28, # specific color for each point 
                                                     #(range between 0 and 1), 28 years of data
             title="Large Earthquakes");

Pandas gave us a default legend again, but the values aren't really meaningful; we want them to correspond to years. Pandas is great for easy defaults, but to control the legend more precisely, we turn to matplotlib directly.

In [ ]:
#matplotlib

# we'll save the output from the plot call so we can use the data later for a legend
pts = plt.scatter(bigones['MAG'], bigones['DEP'], # data
            s=50, # marker size
            cmap="cool", # color scheme: https://matplotlib.org/users/colormaps.html
            c=1.0*(bigones['YEAR']-1983)/28) # specific color for each point (range between 0 and 1)
plt.ylabel("Depth")
plt.xlabel("Magnitude")
plt.title("Large Earthquakes")
clrbar = plt.colorbar(pts, ticks=[1.0/28, 27.0/28]) # data takes values from 1984=1/28, 2010=27/28 (none in 2011)
clrbar.ax.set_yticklabels(['1984', '2010']); # semicolon only needed after final command in cell 
                                             # to suppress printing the object type

## Working with Dates, Aggregating

Make a proper date column (instead of the current multiple columns for each part of the date) in the data frame so that we can plot by date.  Setting the new date column as an index lets us group by month easily.

In [ ]:
df['datetime'] = pd.to_datetime(df[df.columns[range(6)]]) ## first 6 columns are year, month, day, hour, min, sec
df = df.set_index("datetime")

Plot number of earthquakes by month with a bar chart, limit to recent years (the last 48 observations are the last 4 years (4 years * 12 months)

In [ ]:
counts = df.groupby(pd.TimeGrouper(freq='M')).count()['ID'][-48:] # do the aggregation
counts.head()

When we plot this time, we'll need the axes, so we'll use `plt.subplots()` to have it returned.  `plt.subplots()` is a convenience function that returns both figure and axes, instead of doing:

```
fig = plt.figure(figsize=(10, 3.5))
ax = plt.subplot(1,1,1) # or plt.subplot(111) is equivalent
```

In [ ]:
# matplotlib
fig, ax = plt.subplots(figsize=(10, 3.5)) # specify a plot size, use subplots to get axes because we'll need it later
plt.bar(counts.index, counts, width=15) # if we don't set the bar width, some bars might not be visible with this many
ax.xaxis_date()
plt.title("Monthly Earthquake Count for Northern California")
plt.ylabel("Number of Earthquakes");

We could also plot the `counts` object directly:

In [ ]:
#pandas
counts.plot();

In [ ]:
#pandas 
counts.plot(kind="bar");

Plot hourly earthquakes on a busy day

In [ ]:
#matplotlib
df['date'] = pd.to_datetime(df[df.columns[range(3)]])
counts_hourly = df[df['date'] == dt.datetime(1989, 10, 18).date()].resample('1H').count()['ID'] 
# ID above is just to choose a single column
plt.subplots(figsize=(8, 3.5))
plt.bar(counts_hourly.index, counts_hourly);

Hmm, that doesn't look too good.  Fix it.  While we're at it, change the color and bar alignment.  And format the times to be just the hour.

This time, will use the figure, then subplot method, instead of using the subplots function.

In [ ]:
#matplotlib
fig = plt.figure() # just a different way to do it
ax = fig.add_subplot(111) ## saying that there's only one main plot being added to this figure
plt.bar(counts_hourly.index, counts_hourly, width=.03, color='g', align='center')
ax.xaxis.set_major_formatter(dates.DateFormatter('%H'))

Looking better, but we can make it nicer

In [ ]:
#matplotlib
fig, ax = plt.subplots() ## in one line instead of two above, just to show this
plt.bar(counts_hourly.index, counts_hourly, width=.03, color='g', align='center')
ax.xaxis.set_major_formatter(dates.DateFormatter('%H:00'))
#plt.ylim(0, 95) ## could modify ylim manually, but below works even if data changes
ax.margins(0.025) ## control extra space at beginning and end of x-axis
plt.title("Northern California Earthquakes, 18 October 1989")
plt.xlabel("Hour of the Day")
plt.ylabel("Number of Earthquakes");

Save our plot

In [ ]:
plt.savefig("plot1.pdf")

## Layered Plots

More on markers and legends; generate some random values with `randn` and make a histogram.  Add the theoretical distirbution line above it

Info on named colors at https://matplotlib.org/examples/color/named_colors.html

In [ ]:
from scipy.stats import norm
from numpy.random import randn

# keep the values returned from calling the histogram plot to use with the legend and next plot
n, bins, patches = plt.hist(randn(1000), 40, normed=True, color="lightblue")

# add another plot on top of the histogram
# save the value here too so we can use it for the legend
l, = plt.plot(bins, norm.pdf(bins, 0.0, 1.0), 'g--', linewidth=3)
## comma above in l, means unpack the returned tuple to give you just the line object

plt.legend([l, patches[0]], ['theory', 'data']);

## Annotating Plots

In [ ]:
counts = df['date'].value_counts()
counts.plot() ## plot the Series directly
plt.title("Daily Earthquake Count for Northern California")
plt.ylabel("Number of Earthquakes")
plt.text(dt.datetime(1998,3,1), 925, "Big One!"); # use date to position on x-axis

Or with an arrow

In [ ]:
counts = df['date'].value_counts()
counts.plot() ## plot the Series directly
plt.title("Daily Earthquake Count for Northern California")
plt.ylabel("Number of Earthquakes")
plt.annotate("Big One!", xy=(dt.datetime(1998,3,1), 925),
             xytext=(dt.datetime(2000,3,1), 800),
             arrowprops=dict(facecolor='black', shrink=0.05, width=1,
                             headwidth=8));

## Mathematical Functions

Let's make a plot like the following:

![mathplot](https://github.com/nuitrcs/pythonworkshops/raw/master/dataanalysis/visualization/mathfunction.png)

For math text used in the labels, see https://matplotlib.org/users/mathtext.html

In [ ]:
import numpy as np

In [ ]:
# create empty plot
fig = plt.figure(figsize=(4, 5))
ax = fig.add_subplot(111)

# generate data
x = np.arange(-2*np.pi, 2*np.pi, 0.01)
y = np.sin(x)

# deal with bounding box 
# lose top and right
# move bottom and left to the center like a mathematical graph
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position('center')
ax.spines['bottom'].set_position('center')

ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')

# plot the sin wave
ax.plot(x, y,linewidth=3, alpha=0.3)

# plot the reference lines
ax.plot([0, np.pi/2], [1, 1], ls="--", color="green", linewidth=3,alpha=0.5)
ax.plot([np.pi/2, np.pi/2], [1, 0], ls="--", color="red", linewidth=3,alpha=0.5)

# axis limits
ax.set_xlim(-2*np.pi, 2*np.pi)
ax.set_ylim(-1.5, 1.5)

# axis labels
xticker = np.arange(-np.pi-np.pi/2, np.pi+np.pi, np.pi/2)
xlabels = [r"$\frac{-3\pi}{2}$", r"${-\pi}$",r"$\frac{-\pi}{2}$","",r"$\frac{pi}{2}$",r"${\pi}$",r"$\frac{3\pi}{2}$"]

ax.set_xticks(xticker)
ax.set_xticklabels(xlabels, size=17)

yticker = np.arange(-1, 2, 1)
ax.set_yticks(yticker)

# annotation of function line
ax.text(np.pi, 1.1, "y=sin(x)");

## Subplots

We used the subplot function above, but not to create multiple plots.  Let's actually make more than one.  

The marker/line type specification shortcuts used below can be seen at https://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.plot

In [ ]:
# function and arrays to generate some data
def f(t):
    return np.exp(-t) * np.cos(2*np.pi*t)

t1 = np.arange(0.0, 5.0, 0.1)
t2 = np.arange(0.0, 5.0, 0.02)

# create a figure
plt.figure(1)
plt.subplot(211) # 211 = 2 rows, 1 column, first plot
plt.plot(t1, f(t1), 'bo', t2, f(t2), 'k') # bo = blue, o marker

plt.subplot(212) # 212 = 2 rows, 1 column, second plot
plt.plot(t2, np.cos(2*np.pi*t2), 'r--'); # r-- = red, dashed line

## Styling

You can change the default style of matplotlib plots.  A list of available styles:

In [ ]:
print(matplotlib.style.available)

And you can see a gallery of what plots look like in the styles at https://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html

To just affect a single plot, with `with`

In [ ]:
with plt.style.context(('dark_background')):
    pts = plt.scatter(bigones['MAG'], bigones['DEP'], # data
            s=50, # marker size
            cmap="cool", # color scheme: https://matplotlib.org/users/colormaps.html
            c=1.0*(bigones['YEAR']-1983)/28) # specific color for each point (range between 0 and 1)
    plt.ylabel("Depth")
    plt.xlabel("Magnitude")
    plt.title("Large Earthquakes")
    clrbar = plt.colorbar(pts, ticks=[1.0/28, 27.0/28]) # data takes values from 1983=1/28, 2010=27/28 (none in 2011)
    clrbar.ax.set_yticklabels(['1984', '2010'])

In [ ]:
with plt.style.context(('fivethirtyeight')):
    pts = plt.scatter(bigones['MAG'], bigones['DEP'], # data
            s=50, # marker size
            cmap="cool", # color scheme: https://matplotlib.org/users/colormaps.html
            c=1.0*(bigones['YEAR']-1983)/28) # specific color for each point (range between 0 and 1)
    plt.ylabel("Depth")
    plt.xlabel("Magnitude")
    plt.title("Large Earthquakes")
    clrbar = plt.colorbar(pts, ticks=[1.0/28, 27.0/28]) # data takes values from 1983=1/28, 2010=27/28 (none in 2011)
    clrbar.ax.set_yticklabels(['1984', '2010'])

You can also write your own styles: https://matplotlib.org/users/style_sheets.html

## Interactive Plots

Above we turned off the interactive plots in the notebook.  Let's turn them back on to see what they do.

In [ ]:
%matplotlib notebook
plt.figure()
pts = plt.scatter(bigones['MAG'], bigones['DEP'], # data
        s=50, # marker size
        cmap="cool", # color scheme: https://matplotlib.org/users/colormaps.html
        c=1.0*(bigones['YEAR']-1983)/28) # specific color for each point (range between 0 and 1)
plt.ylabel("Depth")
plt.xlabel("Magnitude")
plt.title("Large Earthquakes")
clrbar = plt.colorbar(pts, ticks=[1.0/28, 27.0/28]) # data takes values from 1983=1/28, 2010=27/28 (none in 2011)
clrbar.ax.set_yticklabels(['1984', '2010']);

And now turn it back off:

In [ ]:
%matplotlib inline

# [Seaborn](https://seaborn.pydata.org)
----

[Seaborn](https://seaborn.pydata.org) is one option moving beyond the basic plots.  It's focus is statistical visualization.

Seaborn is not part of the Anaconda distribution, so you'll need to install it.  Copy the command below into a new cell and run it (only the first time you need to use it):

```
!pip3 install seaborn
```

Note that if you're running multiple versions of Python on your system, you might need to install the package from the command line instead into the correct environment.  Via the terminal, if you're using Anaconda, use `conda install seaborn`.

In [ ]:
import seaborn as sns

Re-do a plot from above to see styling change

From before:
![matplotlibversion](https://github.com/nuitrcs/pythonworkshops/raw/master/dataanalysis/visualization/mplcompplot.png)

In [ ]:
sns.set() # turn on Seaborn default style

fig, ax = plt.subplots() ## in one line instead of two above, just to show this
plt.bar(counts_hourly.index, counts_hourly, width=.03, color='g', align='center')
ax.xaxis.set_major_formatter(dates.DateFormatter('%H:00'))
#plt.ylim(0, 95) ## could modify ylim manually, but below works even if data changes
ax.margins(0.025) ## control extra space at beginning and end of x-axis
plt.title("Northern California Earthquakes, 18 October 1989")
plt.xlabel("Hour of the Day")
plt.ylabel("Number of Earthquakes");

Above, the background and grid are different, the font is different, and tick marks and scaling are different from matplotlib defaults.

Other styles/themes are available: https://seaborn.pydata.org/tutorial/aesthetics.html#seaborn-figure-styles

Seaborn also does have nice color palettes built in: http://seaborn.pydata.org/tutorial/color_palettes.html

Note that if you just want styling, some Seaborn styles are now available directly through matplotlib styles like we used above.

But Seaborn is more than styles.  Seaborn also adds some plot types that build on some summary statistics it can compute on data.

## Density/Bivariate Distribution Plots

When we have a lot of data, scatter plots can get difficult to read.  Let's do a density plot instead to explore a bivariate distribution.  Seaborn has several options, both for the density plot and supplementary plots along the axes.  http://seaborn.pydata.org/tutorial/distributions.html#plotting-bivariate-distributions

In [ ]:
with sns.axes_style("white"): #using a different style than the default
    sns.jointplot(x=df['MAG'][:1000], y=df['DEP'][:1000], kind="hex", color="k"); # color k is black

`jointplot` takes care of the overall plot.  If you want more control over the core and axes plots, you can add a density plot and axes plots to an existing matplotlib figure.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
sns.kdeplot(df['MAG'][:1000], df['DEP'][:1000], ax=ax)
sns.rugplot(df['MAG'][:1000], color="g", ax=ax)
sns.rugplot(df['DEP'][:1000], vertical=True, ax=ax);

## Regression and Trend Lines

Seaborn has many functions for visualizing linear relationships and even interaction effect between (or conditional effects of) variables: http://seaborn.pydata.org/tutorial/regression.html

First, a standard regression plot

In [ ]:
sns.regplot(x="MAG", y="DEP", data=df[:1000]);

This includes a 95% confidence interval on the slope coefficient in the plot by default (you can turn it off with the `ci` parameter).  We can also plot the residuals (errors) from a regression to check fit and adherence to model assumptions:  

In [ ]:
sns.residplot("MAG","DEP", df[:1000]);

There are further options for variations like polynomial fits, logistic regression, loess smoothing, and robust regression (eliminates/down-weights outliers).

In addition to `regplot`, there's also the slightly more powerful `lmplot` that let's us condition the slope and intercept on a third variable.

In [ ]:
df['north'] = df['LAT'] > 38 # create a new variable to divide earthquakes by latitude
sns.lmplot(x="MAG", y="DEP", hue="north", data=df.sample(500));